## Contexto y objetivo del proyecto

Este proyecto se centra en la **selección de la región óptima para la perforación de nuevos pozos petroleros**, considerando tanto la rentabilidad esperada como el riesgo financiero asociado. A partir de datos geológicos y de producción histórica, se aplicaron modelos de regresión y técnicas estadísticas para estimar el volumen de reservas y evaluar el beneficio económico potencial de cada región.

El objetivo principal es **identificar la región que maximiza la ganancia esperada manteniendo el riesgo dentro de límites aceptables**, apoyando una toma de decisiones estratégicas basada en datos para la apertura de 200 nuevos pozos petroleros.

### Descargar librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Cargar datos

In [ ]:
def load_data(filepath):
    data = pd.read_csv(filepath)
    print(data.head())
    print()
    data.info()
    print()
    print(data.describe())
    print()
    print('-' * 40)
    return data

data_0 = load_data('/datasets/geo_data_0.csv')
data_1 = load_data('/datasets/geo_data_1.csv')
data_2 = load_data('/datasets/geo_data_2.csv')

Podemos observar que los tipos de datos son correctos, los nombres de las columnas no tienen caracteres raros y no hay valores ausentes. Todo se ve en orden con las columnas "f0", "f1", "f2" y "product", que son las que vamos a usar.

##### Buscar duplicados

In [ ]:
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

No hay filas duplicadas

### Entrenar y probar el modelo para cada región

#### Dividir los datos en conjunto de entrenamiento y validación (proporción: 75/25)

In [ ]:
# Primero, elegimos las columnas "f0", "f1" y "f2" como las características y la columna "product" como la variable objetivo.
#(la columna "id" no es útil en este proyecto)

features = data_0[['f0', 'f1', 'f2']]
target = data_0['product']

# Dividimos los datos en conjuntos de entrenamiento y validación
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features, target, test_size=0.25, random_state=42)

In [ ]:
# creamos y entrenamos el modelo
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)

print('Resultados de data_0')
mean_predicted_0 = predictions_valid_0.mean()
print('Volumen medio de reservas predicho:', mean_predicted_0)

rmse_0 = mean_squared_error(target_valid_0, predictions_valid_0, squared=False)
print('RMSE del modelo:', rmse_0)

In [ ]:
def data_split(data):
    features = data[['f0', 'f1', 'f2']]
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=42)
    return features_train, features_valid, target_train, target_valid

def create_train_predict(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    return model, predictions_valid

def evaluate_model(target_valid, predictions_valid):
    mean_predicted = predictions_valid.mean()
    #print('Volumen medio de reservas predicho:', mean_predicted)
    
    rmse = mean_squared_error(target_valid, predictions_valid, squared=False)
    #print('RMSE del modelo:', rmse)
    return mean_predicted, rmse

def run_pipeline(data):
    features_train, features_valid, target_train, target_valid = data_split(data)
    model, predictions_valid = create_train_predict(features_train, target_train, features_valid)
    mean_predicted, rmse = evaluate_model(target_valid, predictions_valid)
    return model, predictions_valid, mean_predicted, rmse

In [ ]:
model_1, predictions_valid_1, mean_predicted_1, rmse_1 = run_pipeline(data_1)
model_2, predictions_valid_2, mean_predicted_2, rmse_2 = run_pipeline(data_2)

print('Resultados de data_1')
print('Modelo:', model_1)
print('Volumen medio de reservas predicho:', mean_predicted_1)
print('RMSE del modelo:', rmse_1)
print()
print('Resultados de data_2')
print('Modelo:', model_2)
print('Volumen medio de reservas predicho:', mean_predicted_2)
print('RMSE del modelo:', rmse_2)

A pesar de que la regiones en data_0 y data_2 muestran los volúmnes de reservas más altos, parece ser que la opción más confiable es la región en data_1, ya que el modelo muestra aquí un RMSE extremadamente bajo en comparación con las otras dos regiones, volviéndola así la opción más precisa y con el menor nivel de riesgo.

Sin embargo, aunque la tendencia indica que data_1 es la opción más prometedora, hay que tomar en cuenta que esto no garantiza que sea la más rentable, ya que aún necesitamos analizar los resultados económicos y los riesgos que conlleva.

In [ ]:
# Almacenamos los valores necesarios para los cálculos
total_investment = 100000000
total_wells = 200
min_production_per_well_usd = 500000
min_production_per_well_units = 111.1
income_per_unit = 4500

##### Para el cálculo del beneficio, realizamos un estudio de 500 puntos con la selección de los mejores 200 puntos por región.

In [ ]:
# Creamos una función que devuelve el dataset filtrado por los mejores 200 pozos y el valor medio de reservas predicho
def select_top_200(data, model):
    sample = data.sample(n=500, random_state=42)
    features_sample = sample[['f0', 'f1', 'f2']]
    predictions = model.predict(features_sample)
    sample['predictions'] = predictions # creamos una nueva columna para las predicciones de la muestra
    top_200 = sample.sort_values(by='predictions', ascending=False).head(200) # ordenamos por predicción y seleccionamos los primeros 200
    return top_200['predictions'], top_200['predictions'].mean(), top_200

In [ ]:
top_200_predictions_0, top_200_predictions_mean_0, top_200_0 = select_top_200(data_0, model_0)
top_200_predictions_1, top_200_predictions_mean_1, top_200_1 = select_top_200(data_1, model_1)
top_200_predictions_2, top_200_predictions_mean_2, top_200_2 = select_top_200(data_2, model_2)

##### Comparamos el valor medio de reservas predicho con la cantidad mínima de reservas necesarias para evitar pérdidas (111.1 unidades).

In [ ]:
print("Cantidad mínima de reservas necesarias para evitar pérdidas:")
print(111.1)
print('Valor medio de reservas predicho en top_200_0:')
print(top_200_predictions_mean_0)
print()

print("Cantidad mínima de reservas necesarias para evitar pérdidas:")
print(111.1)
print('Valor medio de reservas predicho en top_200_1:')
print(top_200_predictions_mean_1)
print()

print("Cantidad mínima de reservas necesarias para evitar pérdidas:")
print(111.1)
print('Valor medio de reservas predicho en top_200_2:')
print(top_200_predictions_mean_2)

Podemos observar que las tres regiones alcanzan el mínimo de unidades necesarias para evitar pérdidas, lo que quiere decir que, en promedio, cada pozo recuperaría su inversión y generaría ganancias . Hay que destacar que la región 1 tiene el valor medio más alto, lo que la convertiría en la opción más rentable. Por otro lado, la región 2 supera apenas el umbral mínimo, por lo que presenta un mayor riesgo en caso de algún fallo en la predicción.

Para el cálculo del beneficio neto por región, necesitamos restar la inversión de los ingresos; para obtener los ingresos, necesitamos multiplicar el total de reservas reales por el ingreso generado por unidad (4500 usd).

Dicho esto, los pasos serían:

    - Obtener la suma total de reservas reales de los 200 pozos seleccionados por región.
    - Multiplicar dicha suma por 4500 usd.
    - Al resultado obtenido, restarle la inversión.

In [ ]:
def calculate_net_profit(data):
    total_units = data['product'].sum() # suma de reservas reales
    income = total_units * income_per_unit # ingresos totales
    net_profit = income - total_investment # beneficio neto
    return net_profit

In [ ]:
print("Beneficio neto por región en USD:")
net_profit_0 = calculate_net_profit(top_200_0)
print("top_200_0:", round(net_profit_0, 2))
net_profit_1 = calculate_net_profit(top_200_1)
print("top_200_1:", round(net_profit_1, 2))
net_profit_2 = calculate_net_profit(top_200_2)
print("top_200_2:", round(net_profit_2, 2))

Podemos observar que la región que genera mayor beneficio neto es la región 1, con un total de $5,405,081.25 USD. Por esto mismo, hasta el momento, se propone la región 1 para el desarrollo de los pozos petrolíferos, ya que supera por mucho a las otras dos regiones.

Aunque dicha región parezca la más rentable, no hay que olvidar que aún hay que analizar los riesgos potenciales para cada una de las regiones.

### Cálculo de riesgos y ganancias para cada región

In [ ]:
regions = {'region_0':top_200_predictions_0,
           'region_1':top_200_predictions_1,
           'region_2':top_200_predictions_2}

state = np.random.RandomState(42)
mean_net_profit = {}

for name, preds in regions.items():
    values = []
    for i in range(1000):
        subsample = preds.sample(n=200, replace=True, random_state=state)
        net_profit = (subsample.sum() * income_per_unit) - total_investment
        values.append(net_profit)
    
    values = pd.Series(values)
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = (((values < 0).sum()) / len(values)) * 100

    mean_net_profit[name] = mean
    
    print(f'Resultado de {name}')
    print('Beneficio promedio:', round(mean, 2))
    print('Límite inferior del intervalo de confianza:', round(lower, 2))
    print('Límite superior del intervalo de confianza:', round(upper, 2))
    print('Riesgo de pérdida en porcentaje:', risk)
    print()

### Rendimiento sobre la inversión en la región seleccionada

In [ ]:
# Para calcular el roi hay que dividir el beneficio neto promedio entre la inversión total
roi = (mean_net_profit['region_1']) / total_investment
roi_percent = roi * 100 # convertimos el roi a porcentaje
print("Por cada dolar invertido se obtienen:", round(roi, 4), "dólares de ganancia neta")
print("El ROI es de:", round(roi_percent, 2), "%")

### Conclusión

Después de haber realizado el cálculo de riesgos y ganancias potenciales para cada región, así como haber obtenido el intervalo de confianza,  se puede apreciar que la región 2 sería la menos rentable, ya que cuenta con un límite inferior del intervalo muy por debajo de cero, lo que representa un riesgo de pérdida bastante significativo. Por otro lado, las regiones 0 y 1 cuentan con un riesgo prácticamente nulo.

Es evidente que la región 1 es la mejor opción para el desarrollo de pozos petroleros debido a que cuenta con el beneficio neto promedio más alto entre todas las regiones, siendo este de $5,253,001.28 USD.

Además, su intervalo de confianza del 95% va de `$3,069,343.88 USD` a `$7,480,275.6 USD`, lo que quiere decir que en el 95% de las simulaciones, el beneficio neto se encontrará dentro de este rango.

Finalmente, el ROI estimado es del 5.25%, lo que significa que por cada dólar invertido se espera obtener aproximadamente 5 centavos de ganancia neta. Este resultado refuerza la decisión de seleccionar la región 1 para el desarrollo de pozos petroleros, ya que combina un alto beneficio promedio con un riesgo mínimo.